In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import logging
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import chi2_contingency
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
import featuretools as ft
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import confusion_matrix
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from catboost import CatBoostClassifier

In [ ]:
dir_name = '/kaggle/input/forest-cover-type-prediction'
training_file_name = "train.csv"
test_file_name = 'test.csv'


#read data
train_data = pd.read_csv(dir_name + "/" + training_file_name)
test_data = pd.read_csv(dir_name + "/" + test_file_name)
#display(train_data)
#display(test_data)

train_data = train_data.drop("Id", axis=1)
test_data_Id = test_data['Id']
test_data = test_data.drop("Id", axis = 1)

display(train_data)
display(test_data)


# fill in missing values appropriately
#print(train_data.isnull().sum(axis = 0))
#print(test_data.isnull().sum(axis = 0))

In [ ]:
train_data['Road+Fire'] = train_data['Horizontal_Distance_To_Roadways'] + train_data['Horizontal_Distance_To_Fire_Points']
train_data['Road-Fire'] = train_data['Horizontal_Distance_To_Roadways'] - train_data['Horizontal_Distance_To_Fire_Points']
train_data['Road+Hydro'] = train_data['Horizontal_Distance_To_Roadways'] + train_data['Horizontal_Distance_To_Hydrology']
train_data['Road-Hydro'] = train_data['Horizontal_Distance_To_Roadways'] - train_data['Horizontal_Distance_To_Hydrology']
train_data['Hydro+Fire'] = train_data['Horizontal_Distance_To_Hydrology'] + train_data['Horizontal_Distance_To_Fire_Points']
train_data['Hydro-Fire'] = train_data['Horizontal_Distance_To_Hydrology'] - train_data['Horizontal_Distance_To_Fire_Points']

train_data['Road+Fire+Hydro'] = train_data['Horizontal_Distance_To_Roadways']  + train_data['Horizontal_Distance_To_Fire_Points'] + train_data['Horizontal_Distance_To_Hydrology']

train_data['Ele+Road+Fire+Hydro'] = train_data['Elevation'] + train_data['Horizontal_Distance_To_Roadways']  + train_data['Horizontal_Distance_To_Fire_Points'] + train_data['Horizontal_Distance_To_Hydrology']

train_data['Ele+road'] = train_data['Elevation'] + train_data['Horizontal_Distance_To_Roadways']
train_data['Ele-road'] = train_data['Elevation'] - train_data['Horizontal_Distance_To_Roadways']
train_data['Ele+fire'] = train_data['Elevation'] + train_data['Horizontal_Distance_To_Fire_Points']
train_data['Ele-fire'] = train_data['Elevation'] - train_data['Horizontal_Distance_To_Fire_Points']
train_data['Ele+hydro'] = train_data['Elevation'] + train_data['Horizontal_Distance_To_Hydrology']
train_data['Ele-hydro'] = train_data['Elevation'] - train_data['Horizontal_Distance_To_Hydrology']

train_data['distance-to-hydrology'] = (train_data['Horizontal_Distance_To_Hydrology']**2 + train_data['Vertical_Distance_To_Hydrology']**2)**0.5

test_data['Road+Fire'] = test_data['Horizontal_Distance_To_Roadways'] + test_data['Horizontal_Distance_To_Fire_Points']
test_data['Road-Fire'] = test_data['Horizontal_Distance_To_Roadways'] - test_data['Horizontal_Distance_To_Fire_Points']
test_data['Road+Hydro'] = test_data['Horizontal_Distance_To_Roadways'] + test_data['Horizontal_Distance_To_Hydrology']
test_data['Road-Hydro'] = test_data['Horizontal_Distance_To_Roadways'] - test_data['Horizontal_Distance_To_Hydrology']
test_data['Hydro+Fire'] = test_data['Horizontal_Distance_To_Hydrology'] + test_data['Horizontal_Distance_To_Fire_Points']
test_data['Hydro-Fire'] = test_data['Horizontal_Distance_To_Hydrology'] - test_data['Horizontal_Distance_To_Fire_Points']

test_data['Road+Fire+Hydro'] = test_data['Horizontal_Distance_To_Roadways']  + test_data['Horizontal_Distance_To_Fire_Points'] + test_data['Horizontal_Distance_To_Hydrology']

test_data['Ele+Road+Fire+Hydro'] = test_data['Elevation'] + test_data['Horizontal_Distance_To_Roadways']  + test_data['Horizontal_Distance_To_Fire_Points'] + test_data['Horizontal_Distance_To_Hydrology']

test_data['Ele+road'] = test_data['Elevation'] + test_data['Horizontal_Distance_To_Roadways']
test_data['Ele-road'] = test_data['Elevation'] - test_data['Horizontal_Distance_To_Roadways']
test_data['Ele+fire'] = test_data['Elevation'] + test_data['Horizontal_Distance_To_Fire_Points']
test_data['Ele-fire'] = test_data['Elevation'] - test_data['Horizontal_Distance_To_Fire_Points']
test_data['Ele+hydro'] = test_data['Elevation'] + test_data['Horizontal_Distance_To_Hydrology']
test_data['Ele-hydro'] = test_data['Elevation'] - test_data['Horizontal_Distance_To_Hydrology']

test_data['distance-to-hydrology'] = (test_data['Horizontal_Distance_To_Hydrology']**2 + test_data['Vertical_Distance_To_Hydrology']**2)**0.5

display(train_data)
display(test_data)

In [ ]:
'''removed_columns = ['Hillshade_3pm',
 'Vertical_Distance_To_Hydrology',
 'Hillshade_Noon',
 'Wilderness_Area4',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type15',
 'Soil_Type25',
 'Soil_Type11',
 'Soil_Type16',
 'Soil_Type19',
 'Soil_Type28',
 'Soil_Type34']
 
removed_columns = ['Soil_Type7', 'Soil_Type15']

train_data.drop(columns=removed_columns, axis=1, inplace=True)
test_data.drop(columns=removed_columns, axis=1, inplace=True)

display(train_data)
display(test_data)

'''

x = train_data.drop('Cover_Type', axis=1)
y = train_data['Cover_Type']

display(x)
display(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=0)

display(x_train)
display(y_train)
display(x_test)
display(y_test)


In [ ]:
model = LGBMClassifier(eval_metric = 'logloss', feature_fraction = 0.5, max_depth = 20, num_leaves= 1000)
model.fit(x,y)
y_pred = model.predict(test_data)

predictions_test = pd.DataFrame(y_pred)
predictions_test.columns = ['Cover_Type']
display(predictions_test)

result = pd.concat([test_data_Id, predictions_test], axis=1)
display(result)

result.to_csv('lgbm1.csv', index=False)